# Siraj Keras example to generate Nietszche-like text

In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

/home/bill/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = io.open(path, encoding='utf-8').read().lower()
print('corpus length:', len(text))

corpus length: 600893


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


In [4]:
# cut the text into semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 200285


In [5]:
# make input vectors from the previously massaged data
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [6]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

Build model...


In [7]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [8]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [9]:
def on_epoch_end(epoch, logs):
    # Function invoked at the end of each epoch. Prints generated text.
    print()
    print('------ Generating text after Epoch: %d' % epoch)
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [10]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [11]:
model.fit(x, y,
         batch_size=128,
         epochs=60,
         callbacks=[print_callback])

Epoch 1/60
200285/200285 [==============================] - 355s 2ms/step - loss: 3.2492

------ Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "hat the new
tendencies which they oppose"
hat the new
tendencies which they oppose    en  s      e      e     e     e  ee e  ti  e  e e     t e  e  e   e  e ee     e             e    eeme  e e e     ee n  
   
ete         e  t        e     e ee  e     te  e        
 e  ee       e     e  e
            ehe              e    e
   i  ee    eu e  e    e      eee   e     
          
  e     e        ee    e  ee      
  e  e  e   e      e           e  e        ee   e    e i    e      
----- diversity: 0.5
----- Generating with seed: "hat the new
tendencies which they oppose"
hat the new
tendencies which they opposeog eeoueea
 e   etyon
 i amuee e na i ete 
  
o e nh ie
a  t e 
e eu, 
 
 ti r
 m
 teee n upn
i puec e toio  eoom    u    e u gieii e tm tr  uerii 


e   itp e
ot eui  nee
u 
et  v ee 
eue  a eii

  
ee  uii

/home/bill/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


t tll llt ttltlt tt lttlttttl tl tl ll tl t ll   tllttttttltllttt ltlllltlttltt tlt lllttttt llll tl tl tttl ttttl t  ltett httttltt t t t ttttlltt  tlttlttllt llt tt ttl ttttt  lllltttl li  t  lt    ltlttl ttt ttttttltll ltt  l ttttt  tttltttttltlttt ttt tl tlttltitttdtttltt  llll tt t  t tltllttt t tt    lll ltttlt t t  lttteltltltt ttttttltttt tt tttlt ttltlltt ltllt ttllttl tltttlll
----- diversity: 0.5
----- Generating with seed: "of what use is it for
nimble, everyday i"
of what use is it for
nimble, everyday itlel r edtih t llttdtle stt thrt lteltrltllttdtlnltst     l n l ttltlhtt rtlin lt
  l llt d  etltl)l l ertli t lt
ltll   cdt ttsltt tdtsl l lle ttt tmttth uilhid ll lhll em itlt,sh  l ilt lil tti llttlhlt  lttlre tttt lsltttte
rrolll tn   lon tlei d drlltdtllth tllltmtltl ti tlt tl lnttlt te tlltltteeeio ttett dltltcltuet  itrtth ddrlltvtltlslllthllttilteetttnl tlelsl dlhlettt ihtth ll  t  t lodto
----- diversity: 1.0
----- Generating with seed: "of what use is it for
nimbl

mavvlosefsanslilmashiwrtatse,te .oaryettssttei olalotnitmrlair rcolsit yrotrltryo.yitiore1 lyitlirersrcrtssoilry ?isa esltlitihairqiirl
----- diversity: 1.2
----- Generating with seed: " as much as he pleases, he is not by
nat"
 as much as he pleases, he is not by
nat tv :rllynuroybovtoday rcol.lryl.ayretr.luiwaolyayoaltirytryieprtd timoahynsotsro.ierswoirnyrhliioiryaaarsf retu wy r ter rnph oatrylslyyo euia ritdsti eivrre lunryny
san irrrtrtaoiyrioubraoto rw
yei,otmcyotwrornrrtoorhtot iarl
noiiiaoruallryriinorwr rrmhtlaiiirulbilyita tu1ynoyr:
ptmrrowiil:yoiorr
b
 .aeofiirrlmgm trlrtog1pars ayief vynyeral
irtilerserl for.leeln  iittyttos yeeshtsivslt oattgcrto
Epoch 6/60
200285/200285 [==============================] - 385s 2ms/step - loss: 3.2549

------ Generating text after Epoch: 5
----- diversity: 0.2
----- Generating with seed: "hina at
present, where the distinction o"
hina at
present, where the distinction odddeeeedddeeeedddddededddddddededdddeeddeeedtredddddeedddeeddeeddddddee

ges of tempo, in which the ancient publia eueroroiraarriririr nararroaarru
irir orierarraamrirar"raraenraaaerao
earorueaararaa rraa orhirnroriararuiiaerriiiarnuarr
raarrraraarrruraoiarraai uoe roarrariaae rr
nmrrrraerrirareraaraarioanrrrayeaeaaerraaiaraaiai rrraenrauri rioer rrierrrorao aeaoeorraroaahu roeaiaa rai riurrar
 eraoe enarieearrearrrrooaiaiixrr rrerr
e aaire rarruyrirrrrreraimera aeieoa  rrara rrraeueotraoeruroeria auraarri a
----- diversity: 1.0
----- Generating with seed: "ges of tempo, in which the ancient publi"
ges of tempo, in which the ancient publir eiao"oraahwaat 
nefaikh h itiarraeeaonsiciearyrgrrylreerreruioireiiafyiaerireameiomto riee
mfsirarwibrwtfrajo oev umteilreifin  roiasua laa aaaeoanwrrharofa uhu ramexiraaeao riooudni
norirraauermma riiru
a(iuaiifanrf ronmaeiahayara irnwrartiaioranarurf lry
no mc urtera brmmrei!eaa oai,iuer-or  derhh
e
w a aihu a ru"m nefne grraoeum u ameeneesrrirlio glieun xuonlaiyfihuraaor raruairw froin n
oi
r
----- diversity: 1.2
-----

...

b inieu;i    ifnismihi   ligomie  iii einifw  nkiioisi, i n a-tr ne   mnsr ieiaani   inhwiiiin w,ifiiicibopnoisiioini  f  exl- ay u , ii ihpieu!  e
Epoch 45/60
200285/200285 [==============================] - 381s 2ms/step - loss: 3.2641

------ Generating text after Epoch: 44
----- diversity: 0.2
----- Generating with seed: "ing arbitrary in the fact that the
philo"
ing arbitrary in the fact that the
philo   i   ai oeeaioo  ii ie a  ioe iiii oeiloe  ao oeh ii   l   ni ii i iia i i      ao ia i  i aao   iioiieaaa  a     aiao aoie i i i il  i oi eiili      i  ai  iaoieiiia  a a   e i aia i ai la     eoia iiia  eoaaiiioa   e  eho e aae iiiao ae  a il     a ea      i  i hail a ei    a ao i  aii aieao  aiiiaiiiieo  io e     ieoili ii    eiaaiaaei i l  ai eeai  l  i  ieiii     i oeili  ii l iee e i  aoia
----- diversity: 0.5
----- Generating with seed: "ing arbitrary in the fact that the
philo"
ing arbitrary in the fact that the
philo iaaaaem ieeisliie  eoi eo tinal  eeo illv ooona eooalai 

without efforteaeam e,efmy ,veeteefgaagmlomemleeolge gemefeeatmgeeemeaeefa feeaffey eameofc" eeemeese n:i ieetmeeette ihef  ahmmegflgeealneemfmeml tffealeiuetomg reeeeegseoemlemeeeyemhe  meveeefoeomm tlmem eeeem ef,mbe ueemmtfmleee eeee kmeeememftefpeee innc meel ueemofeemhtemtlmcffeea geyelte rteeeeeefefmeefoeetoel eh acmsdsftlfceemeeeiaueemgmtaem o  leaeemeah teoemgmyoefhtlme.ltefheef etff teeeeeecfyaeee meee
----- diversity: 1.2
----- Generating with seed: "vine levity, an "upwards"
without effort"
vine levity, an "upwards"
without effortooeetoe-.tfmsemoegl,mfefffo,meleeero efem eafe!lemgifmut eweeleteee egemgrm,emopeyeemhaehmeemlaeagem:aeeaet.memetffe ufaomul"lmoaoe
ee oamefhyealutftefffmigfem?eedihaeleeyueeesfeeledole-t"ag. tfmf,th"eemlpmeeinefe=l f-feelemeatrh emffsmlocet,eegeeffeegfleeftgemg tieefioenmtohiee:eef o dflueamgnryle eee,ef rsth eplfmlieimaeyehtnemmteosofuy,nfeshfemacfeete
leneewmmfmeeaffofttffo  mlef easemetemgou.m
Epoch 49/60
200285/200285 [=========================

and coloured, that by virtue       t          t      t   ttt                      t      ta      t   t    e                           e                t                t                  ttt         t                       e            t                  t     e                 t    e           e                              e        t       t                  t                 t   t                   e e   t  e              
----- diversity: 0.5
----- Generating with seed: "hly imbued
and coloured, that by virtue "
hly imbued
and coloured, that by virtue e s ee tt  estt t  ttt o t e tae   tsea tite  te ttr    ce   ts i  te a es te t  a  e  e  te   ea   s      te   tt      e  a a s      eti t  ftnte tes s         e  f  tt sa i   iett e  it  aa e  ee   e  ttai at  tt  t   iaet a et    s s eh tttat e 
 the t   ei s  a te eteea f  t     t ts  a e a ate hr  tte t eh  ps ee       s  te       e  et  t  rse    tee   e e   ar  etfttta   f tah a eehn t   e 
----- diversity: 1.0
----- Generating

ol:f maothl  stgie duer oayoi

ahuiflal lfollir t 
af
roeolhthasflroao ollh a ll

f
i
 tiorrf,ui o
oob lurh rhwll
t-r
----- diversity: 1.2
----- Generating with seed: "--is there perhaps
some enigma therein? "
--is there perhaps
some enigma therein? rrohfa rafoabile:lo-ot
aarblcs swaaltov
fits darlw fy
elc tolf r
fo shr ltfdfa:
aooeti:houxllfamvhrrwscklmrb
wwaftaefrflbrdp
faoriawpya adl olat r  lwbd
ili
ot lferbard yoof ,k:ibitars
o dm
aeo oumbl :a
o a
anl
wm rs 
i auyuety.f  r"rwowowaoeyalt"amic
ohfuoothpa idnsrfr
rirrawimoritiuw 
som 

a l
l-fc felllr
 he a  bo rf  l tl
oasohrdh wto-trat ythsalhehs y tloor
lrtwr  a
 tlslii" 
rwtrat -hli,t;m
Epoch 56/60
200285/200285 [==============================] - 380s 2ms/step - loss: 3.2629

------ Generating text after Epoch: 55
----- diversity: 0.2
----- Generating with seed: "ow it. how strangely pious for our taste"
ow it. how strangely pious for our tasteh hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhshhhhhhhhhhhhhhhhhh

itself.-tttewletwt tnt te te elt nttt tsnnrt t  tttnetrt e   tttentteerttt tetetlnrwte tt teltt  teenneett ne 
e etelltett nmetertrt oett m
eetttttetnt nttrt rettnlenntte tnttetntettent nt   tetnttne  e ttl tt titr senlett
tte elttl  t  lmltli tt pentlttetee t  te t t ettettenlrttreet metese ttntwweeerntt ttn  tetetetestte
rttett t
eettteee rt netsteetlr t
lltteeetltertetl    trtlttttn et tltents
tttttren
----- diversity: 1.0
----- Generating with seed: "he noble soul has reverence for
itself.-"
he noble soul has reverence for
itself.-e oiryide stnltteerteereem
gntlnma nssewe
lfl rr
eeta
iectp iwn ttu eornle esg 


e omdrtt p er rcealetvte lle tmtntt eo mltrsnt lyetu
temrnrst-tceeae  t tnlmttttw l 
tlelh
.tcewe idtrttwmneuoel
tett llrn csdsae c
ew snn

 tetree  em rtnrw s-nnfmt sn sntmw  tmrentit
eetts nsuttt
nrte ttetm tnltrmrpt 
ereetdtvc-ehntlemf e   e ietttttele mec ttaeectnrlrfwtre
t ttew 
srt rs
rt tssltetttwmrlttl  nu"  
----- diversity: 1.2
----- Generating with seed: "he noble